In [1]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.expr.types import BooleanValue, Table

from mismo.block import Blocking, block_on_arrays
from mismo.examples import load_patents

ibis.options.interactive = True

In [2]:
patents = load_patents()
patents

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

In [3]:
from mismo.clean.strings import norm_whitespace


def clean_names(names):
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


def bin_coordinate(coord):
    return (coord.round(1) * 10).cast("int16")


cleaned = patents
cleaned = cleaned.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=clean_names(_.name),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=_.coauthors.upper().split("**").map(norm_whitespace).sort(),
    classes=_.classes.upper().split("**").map(norm_whitespace).sort(),
)
cleaned = cleaned.mutate(
    name_tokens=_.name_cleaned.split(" ").map(norm_whitespace).sort(),
    name_first3=_.name_cleaned[0:3],
    latitude_binned=bin_coordinate(_.latitude),
    longitude_binned=bin_coordinate(_.longitude),
)
cleaned

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned             ┃ latitude ┃ longitude ┃ coauthors                                                                           ┃ classes                  ┃ name_tokens                 ┃ name_first3 ┃ latitude_hash ┃ longitude_hash ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ string                   │ float64  │ float64   │ array<string>                                                                       │ array<string>            │ array<string>               │ string      │ int16         │ int16          │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────────┼──────────┼───────────┼─────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────┼─────────────────────────────┼─────────────┼───────────────┼────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES INC │      nan │       nan │ ['DAVID E SNYDER', 'KONINK PHILIPS ELECTRONICS N V', ... +1]                        │ ['A61B', 'A61N']         │ ['AGILENT', 'INC', ... +1]  │ AGI         │          NULL │           NULL │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL NV            │      nan │       nan │ ['ALBERT BORREMAN', 'ALBERTO SLIKTA', ... +11]                                      │ ['B01L', 'C11D', ... +3] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │          NULL │           NULL │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL NV            │      nan │       nan │ ['HANS OSKARSSON', 'JOHANNA CHRISTINA SPEELMAN', ... +3]                            │ ['B01F', 'C09K', ... +2] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │          NULL │           NULL │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['EVAN JOHN STANBURY', 'GUENTER KOCHSMEIER', ... +2]                                │ ['G02B', 'G04G', ... +2] │ ['ALCATEL', 'NV']           │ ALC         │           524 │             49 │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['CHRISTIAN LE SERGENT', 'DUANE LYNN MORTENSEN', ... +2]                            │ ['B05D', 'C03B', ... +6] │ ['ALCATEL', 'NV']           │ ALC         │           524 │             49 │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │      nan │       nan │ ['JEAN MICHEL GABRIAGUES', 'MICHEL SOTOM', ... +1]                                  │ ['H01S', 'H04B', ... +1] │ ['ALCATEL', 'NV']           │ ALC         │          NULL │           NULL │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA NV          │      nan │       nan │ ['CANON RES CENT EURO', 'LEE RICKLER', ... +3]                                      │ ['G06F']                 │ ['CANON', 'EUROPA', ... +1] │ CAN         │          NULL │           NULL │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │ CANON EUROPA NV          │      nan │       nan │ ['ADAM MICHAEL BA

In [4]:
def name_rule(left: Table, right: Table) -> BooleanValue:
    return left.name_cleaned == right.name_cleaned


def coord_rules(left: Table, right: Table) -> BooleanValue:
    return [
        (
            (left.latitude_binned == right.latitude_binned)
            & (left.longitude_binned == right.longitude_binned)
        ),
        (
            (left.latitude_binned == right.latitude_binned)
            & left.longitude_binned.isnull()
            & right.longitude_binned.isnull()
        ),
        (
            left.latitude_binned.isnull()
            & right.latitude_binned.isnull()
            & (left.longitude_binned == right.longitude_binned)
        ),
    ]


rules = [
    name_rule,
    coord_rules,
    block_on_arrays("name_tokens", "name_tokens"),
    block_on_arrays("coauthors", "classes"),
]


cleaned = cleaned.cache()
left = cleaned
right = cleaned.view()
blocking = Blocking(left, right, rules)
blocked = blocking.blocked.cache()
blocked

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                ┃ coauthors_l                                                       ┃ coauthors_r                               ┃ label_true_l ┃ label_true_r ┃ latitude_hash_l ┃ latitude_hash_r ┃ latitude_l ┃ latitude_r ┃ longitude_hash_l ┃ longitude_hash_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                                     ┃ name_cleaned_r        ┃ name_first3_l ┃ name_first3_r ┃ name_l                                                 ┃ name_r                  ┃ name_tokens_l                          ┃ name_tokens_r                ┃ name_true_l                                     ┃ name_true_r        ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>            │ array<string>                                                     │ array<string>                             │ int64        │ int64        │ int16           │ int16           │ float64    │ float64    │ int16            │ int16            │ float64     │ float64     │ string                                             │ string                │ string        │ string        │ string                                                 │ string                  │ array<string>                          │ array<string>                │ string                                          │ string             │
├─────────────┼─────────────┼───────────────────────────┼──────────────────────────┼───────────────────────────────────────────────────────────────────┼───────────────────────────────────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┼────────────┼────────────┼──────────────────┼──────────────────┼─────────────┼─────────────┼────────────────────────────────────────────────────┼───────────────────────┼───────────────┼───────────────┼────────────────────────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────┼──────────────────────────────┼─────────────────────────────────────────────────┼────────────────────┤
│    18321837 │    32621608 │ ['C12Q', 'G06F']          │ ['H01L']                 │ ['SCHAFFER J. DAVID', 'SIMPSON MARK R']                           │ ['MIRHAMED ARASH', 'PRIMA JENS', ... +3]  │     26842439 │     32634657 │             525 │             525 │      52.50 │      52.50 │               58 │               58 │    5.750000 │    5.750000 │ KONINKLIJKE PHILLIPS ELECTRONICS NV                │ STMICROELECTRONICS NV │ KON           │ STM           │ Koninklijke Phillips Electronics N.V.                  │ STMicroelectronics N.V. │ ['E

In [5]:
blocked.count()

2337943

In [6]:
from mismo.compare import Comparison, ComparisonLevel, array_overlap_norm, exact_level
from mismo.compare.fs import FellegiSunterComparer

exact = exact_level("name")
almost_level = exact_level("name_first3")
name_comparison = Comparison(name="Name", levels=[exact, almost_level])

classes_exact_match = exact_level("classes")
classes_50_percent_overlap = ComparisonLevel(
    name="Classes 50% overlap",
    condition=lambda table: array_overlap_norm(table["classes_l"], table["classes_r"])
    >= 0.5,
)
classes_comparison = Comparison(
    name="Classes",
    levels=[classes_exact_match, classes_50_percent_overlap],
)

guess_n_matches_per_record = 3
prior = guess_n_matches_per_record / patents.count().execute()
fs = FellegiSunterComparer([name_comparison, classes_comparison], prior=prior)
fs.comparisons

[FSComparison(comparison=Comparison(name='Name', levels=[ComparisonLevel(name='exact_name', condition=<function exact.<locals>.equals at 0x1326c28e0>, description='Exact match on `name`'), ComparisonLevel(name='exact_name_first3', condition=<function exact.<locals>.equals at 0x10a849440>, description='Exact match on `name_first3`')], description=None), weights=None),
 FSComparison(comparison=Comparison(name='Classes', levels=[ComparisonLevel(name='exact_classes', condition=<function exact.<locals>.equals at 0x1332480e0>, description='Exact match on `classes`'), ComparisonLevel(name='Classes 50% overlap', condition=<function <lambda> at 0x13324b380>, description='Class 50% overlap')], description=None), weights=None)]

In [7]:
# fs_trained = fs.trained(left, right, seed=42)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
# comparisons = fs_trained.compare(blocked)
# comparisons

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                ┃ coauthors_l                                                       ┃ coauthors_r                               ┃ label_true_l ┃ label_true_r ┃ latitude_hash_l ┃ latitude_hash_r ┃ latitude_l ┃ latitude_r ┃ longitude_hash_l ┃ longitude_hash_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                                     ┃ name_cleaned_r        ┃ name_first3_l ┃ name_first3_r ┃ name_l                                                 ┃ name_r                  ┃ name_tokens_l                          ┃ name_tokens_r                ┃ name_true_l                                     ┃ name_true_r        ┃ Name_cmp ┃ Name_bf  ┃ Classes_cmp         ┃ Classes_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>            │ array<string>                                                     │ array<string>                             │ int64        │ int64        │ int16           │ int16           │ float64    │ float64    │ int16            │ int16            │ float64     │ float64     │ string                                             │ string                │ string        │ string        │ string                                                 │ string                  │ array<string>                          │ array<string>                │ string                                          │ string             │ string   │ float64  │ string              │ float64    │ float64  │
├─────────────┼─────────────┼───────────────────────────┼──────────────────────────┼───────────────────────────────────────────────────────────────────┼───────────────────────────────────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┼────────────┼────────────┼──────────────────┼──────────────────┼─────────────┼─────────────┼────────────────────────────────────────────────────┼───────────────────────┼───────────────┼───────────────┼────────────────────────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────┼──────────────────────────────┼─────────────────────────────────────────────────┼────────────────────┼──────────┼──────────┼─────────────────────┼────────────┼──────────┤
│    18321837 │    32621608 │ ['C12Q', 'G06F']          │ ['H01L']                 │ ['SCHAFFER J. DAVID', 'SIMPSON MARK R']                           │ ['MIRHAMED ARASH', 'PRIMA JENS', ... +3]  │     26842439 

In [10]:
# comparison_vectors = comparisons.distinct(on=["Name_cmp", "Classes_cmp"])
# comparison_vectors = comparison_vectors.cache()
# comparison_vectors

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                 ┃ coauthors_l                                                                         ┃ coauthors_r                                                                   ┃ label_true_l ┃ label_true_r ┃ latitude_hash_l ┃ latitude_hash_r ┃ latitude_l ┃ latitude_r ┃ longitude_hash_l ┃ longitude_hash_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                     ┃ name_cleaned_r                      ┃ name_first3_l ┃ name_first3_r ┃ name_l                               ┃ name_r                                ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                ┃ name_true_r                ┃ Name_cmp          ┃ Name_bf  ┃ Classes_cmp         ┃ Classes_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>             │ array<string>                                                                       │ array<string>                                                                 │ int64        │ int64        │ int16           │ int16           │ float64    │ float64    │ int16            │ int16            │ float64     │ float64     │ string                             │ string                              │ string        │ string        │ string                               │ string                                │ array<string>                          │ array<string>                          │ string                     │ string                     │ string            │ float64  │ string              │ float64    │ float64  │
├─────────────┼─────────────┼───────────────────────────┼───────────────────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┼────────────┼────────────┼──────────────────┼──────────────────┼─────────────┼─────────────┼────────────────────────────────────┼─────────────────────────────────────┼───────────────┼───────────────┼──────────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────┼────────────────────────────┼─────────────

In [11]:
# comparison_vectors.order_by("bf")

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                 ┃ coauthors_l                                                                         ┃ coauthors_r                                                                   ┃ label_true_l ┃ label_true_r ┃ latitude_hash_l ┃ latitude_hash_r ┃ latitude_l ┃ latitude_r ┃ longitude_hash_l ┃ longitude_hash_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                     ┃ name_cleaned_r                      ┃ name_first3_l ┃ name_first3_r ┃ name_l                               ┃ name_r                                ┃ name_tokens_l                          ┃ name_tokens_r                          ┃ name_true_l                ┃ name_true_r                ┃ Name_cmp          ┃ Name_bf  ┃ Classes_cmp         ┃ Classes_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>             │ array<string>                                                                       │ array<string>                                                                 │ int64        │ int64        │ int16           │ int16           │ float64    │ float64    │ int16            │ int16            │ float64     │ float64     │ string                             │ string                              │ string        │ string        │ string                               │ string                                │ array<string>                          │ array<string>                          │ string                     │ string                     │ string            │ float64  │ string              │ float64    │ float64  │
├─────────────┼─────────────┼───────────────────────────┼───────────────────────────┼─────────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────┼──────────────┼──────────────┼─────────────────┼─────────────────┼────────────┼────────────┼──────────────────┼──────────────────┼─────────────┼─────────────┼────────────────────────────────────┼─────────────────────────────────────┼───────────────┼───────────────┼──────────────────────────────────────┼───────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────────────────┼────────────────────────────┼────────────────────────────┼─────────────